In [ ]:
#Import dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
import sqlite3
import pandas as pd

#Connect to sqlite
connection = sqlite3.connect(":memory:")

cursor = connection.cursor()

#Read in sql file
sql_file = open("living_in_nashville.sql", encoding="utf8")
sql_as_string = sql_file.read()
cursor.executescript(sql_as_string)


In [ ]:
#Create housing DataFrame
housing_list = []
for row in cursor.execute("SELECT * FROM Nashville_Housing_Data"):
    housing_list.append(row)

#housing_df
housing_df = pd.DataFrame(housing_list)
housing_df.columns = ['Unnamed:0',"ParcelID","LandUse","PropertyAddress","Suite/Condo#","PropertyCity","ZipCode","SaleDate","SalePrice","LegalReference","SoldAsVacant","MultipleParcelsInvolvedinSale","OwnerName","Address","City","State","Acreage","TaxDistrict","Neighborhood","image","LandValue","BuildingValue","TotalValue","FinishedArea","FoundationType","YearBuilt","ExteriorWall","Grade","Bedrooms","FullBath","HalfBath"]
housing_df.head()


In [ ]:
# Create Parks DataFrame
parks_list = []
for row in cursor.execute("SELECT * FROM Park_Locations"):
    parks_list.append(row)

parks_df = pd.DataFrame(parks_list)
parks_df.columns= ["ParkName","ZipCodes","Acres","CommunityCenter","NatureCenter","Playground","ADAAccessible","RestroomsAvailable","DogPark","BaseballFields","BasketballCourts","SoccerFields","Football&Multi-purposeFields","TennisCourts","DiscGolf","SwimmingPool","GolfCourse","Walk&JogPaths","HistoricFeatures","MappedLocation","Lat","Lng"]
parks_df=parks_df.replace({'Yes':True,'No':False})
parks_df.head()

In [ ]:
#Create list for of whether Walk and Jog is True
walk_list_per_zip = parks_df.groupby('ZipCodes')['Walk&JogPaths'].apply(list)
walk_per_zip = []
for code in walk_list_per_zip:
    if True in code:
        walk_per_zip.append(True)
    else: walk_per_zip.append(False)
walk_per_zip

In [ ]:
#Create list for baseball
baseball_per_zip = parks_df.groupby('ZipCodes')['BaseballFields'].apply(list)
baseball_list = []
for code in baseball_per_zip:
    if True in code:
        baseball_list.append(True)
    else: baseball_list.append(False)
#Create list for basketball
basketball_per_zip = parks_df.groupby('ZipCodes')['BasketballCourts'].apply(list)
basketball_list = []
for code in basketball_per_zip:
    if True in code:
        basketball_list.append(True)
    else: basketball_list.append(False)
#Create list for Soccer
soccer_per_zip = parks_df.groupby('ZipCodes')['SoccerFields'].apply(list)
soccer_list = []
for code in soccer_per_zip:
    if True in code:
        soccer_list.append(True)
    else: soccer_list.append(False)
#Create list for Football
football_per_zip = parks_df.groupby('ZipCodes')['Football&Multi-purposeFields'].apply(list)
football_list = []
for code in football_per_zip:
    if True in code:
        football_list.append(True)
    else: football_list.append(False)
#Create list for Tennis
tennis_per_zip = parks_df.groupby('ZipCodes')['TennisCourts'].apply(list)
tennis_list = []
for code in tennis_per_zip:
    if True in code:
        tennis_list.append(True)
    else: tennis_list.append(False)

In [ ]:
#Zip together individual sport lists
sports_tuples = list(zip(baseball_list,basketball_list,soccer_list,football_list,tennis_list))
sports_tuples

In [ ]:
#Create list for whether any of the sports lists contain true in each zip
sports_list = []
for row in sports_tuples:
    if True in row:
        sports_list.append(True)
    else: sports_list.append(False)

sports_list

In [ ]:
#group by zip code and find if there is a yes in ADA column

ada_list_per_zip = parks_df.groupby('ZipCodes')['ADAAccessible'].apply(list)
ada_per_zip = []
for code in ada_list_per_zip:
    if True in code:
        ada_per_zip.append(True)
    else: ada_per_zip.append(False)
ada_per_zip

In [ ]:
# Create parks per zip code variable
parks_per_zip = parks_df.groupby('ZipCodes').size()
parks_per_zip

In [ ]:
# Create variable for total acres of parks per zip code
acres_per_zip = parks_df.groupby('ZipCodes').sum()['Acres']
acres_per_zip

In [ ]:
#Create new DataFrame with created variables
parks_new_tuples = list(zip(parks_per_zip, acres_per_zip, ada_per_zip, sports_list, walk_per_zip))
parks_new_df = pd.DataFrame(parks_new_tuples, columns=["Parks Per Zip", "Acres of Parks Per Zip", "ADA Park in Zip", "Sports Fields in Zip", "Walk Path in Zip"], index=parks_per_zip.index)
parks_new_df

In [ ]:
# Merge new parks df with housing df
new_housing_df = housing_df.merge(parks_new_df, left_on="ZipCode", right_on="ZipCodes")
new_housing_df.head()

In [ ]:
new_housing_df = new_housing_df.drop(columns=["ParcelID", "LandUse","PropertyAddress","Suite/Condo#","Unnamed:0","PropertyCity","SaleDate","LegalReference","ExteriorWall","Grade","SoldAsVacant","MultipleParcelsInvolvedinSale","OwnerName","Address","City","State", "FoundationType","TaxDistrict","image","Neighborhood"])


In [ ]:
new_housing_df = new_housing_df.dropna()
new_housing_df

In [ ]:
# 5. Export the Dataframe as a new CSV file without the index.
new_housing_df.to_csv('./Cleaned_Data/new_housing_final.csv', index=False)

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
from path import Path
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report



In [ ]:
plt.scatter(new_housing_df.YearBuilt, new_housing_df.SalePrice)

In [ ]:
plt.scatter(new_housing_df.FinishedArea, new_housing_df.SalePrice)
plt.show

In [ ]:
plt.scatter(new_housing_df.Acreage, new_housing_df.SalePrice)

In [ ]:
plt.scatter(new_housing_df.Bedrooms, new_housing_df.SalePrice)

In [ ]:
# Split our preprocessed data into our features and target arrays
y = new_housing_df['SalePrice'].values
X = new_housing_df.drop('SalePrice',1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
#Create a linear regression model 
from sklearn.linear_model import LinearRegression
model = LinearRegression()
# fit the model
model.fit(X, y)
# get importance
importance = model.coef_

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Fitting our model with all our features in X
model.fit(X, y)

score = model.score(X, y)
print(f"R2 Score: {score}")

In [ ]:
predictions = model.predict(X)

In [ ]:
print(f"Training Data Score: {model.score(X_train, y_train)}")
print(f"Testing Data Score: {model.score(X_test, y_test)}")

In [ ]:
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
# END OF PROJECT

In [ ]:
#Create Schools DataFrame
schools_list = []
for row in cursor.execute("SELECT * FROM MNPS_Enrollment_Data"):
    schools_list.append(row)


schools_df = pd.DataFrame(schools_list)
schools_df.columns = ["SchoolLevel","SchoolName","ZipCode","Rank","Greatschoolsrating","TotalEnrollment","AmericanIndianorAlaskaNative","Asian","BlackorAfricanAmerican","Hispanic/Latino","NativeHawaiianorOtherPacificIslander","White","StudentswithDisabilities"]
schools_df.head()

In [ ]:
#Create Schools DataFrame
restaurant_list = []
for row in cursor.execute("SELECT * FROM Nashville_Restaurants"):
    restaurant_list.append(row)


restaurant_df = pd.DataFrame(restaurant_list)
restaurant_df.columns = ["Restaurant Name","Neighborhood","Address","ZIP Code","Description"]
restaurant_df.head()

In [ ]:
restaurant_per_zip = restaurant_df.groupby('ZIP Code').size()
restaurant_per_zip

In [ ]:
new_restaurants_df = pd.DataFrame(restaurant_per_zip, columns=["RestaurantsPerZip"], index=restaurant_per_zip.index)
new_restaurants_df 